## cross_val_score

In [52]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris
import numpy as np

iris = load_iris()
dt_clf = DecisionTreeClassifier(random_state=156)

data = iris.data
target = iris.target

# 알고리즘은 결정트리, 성능 지표는 정확도, 교차 검증 세트는 3개
# 분류 - 당연히 Stratified K-Fold 사용
# 회귀는 값 분포가 연속형이기 때문에 K-Fold 적용불가
scores = cross_val_score(dt_clf, data, target, scoring='accuracy', cv=3)
print('교차 검증별 정확도: ', np.round(scores, 4))
print('평균 검증 정확도: ', np.round(np.mean(scores), 4))

교차 검증별 정확도:  [0.98 0.94 0.98]
평균 검증 정확도:  0.9667


## GridSearchCV

In [53]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score

iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=121)

dtree = DecisionTreeClassifier()

# 결정 트리 알고리즘에 쓰이는 파라미터들
parameters = {
    'max_depth': [1, 2, 3],
    'min_samples_split': [2, 3]
}

In [55]:
import pandas as pd

# param_grid의 하이퍼 파라미터들을 3개의 train, test set fold로 나눠서 테스트 수행
# refit이 True면 가장 좋은 파라미터 설정으로 재학습
grid_dtree = GridSearchCV(dtree, param_grid=parameters, cv=3, refit=True, return_train_score=True)

# 붓꽃 학습 데이터로 하이퍼 파라미터들 순차적으로 학습/평가
grid_dtree.fit(X_train, y_train)

scores_df = pd.DataFrame(grid_dtree.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score', 'split0_test_score', 'split1_test_score', 'split2_test_score']]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.700000,5,0.700,0.7,0.70
1,"{'max_depth': 1, 'min_samples_split': 3}",0.700000,5,0.700,0.7,0.70
2,"{'max_depth': 2, 'min_samples_split': 2}",0.958333,3,0.925,1.0,0.95
3,"{'max_depth': 2, 'min_samples_split': 3}",0.958333,3,0.925,1.0,0.95
4,"{'max_depth': 3, 'min_samples_split': 2}",0.975000,1,0.975,1.0,0.95
5,"{'max_depth': 3, 'min_samples_split': 3}",0.975000,1,0.975,1.0,0.95


In [56]:
grid_dtree.cv_results_

{'mean_fit_time': array([0.00070731, 0.00065533, 0.00023103, 0.00025058, 0.00025964,
        0.00022578]),
 'std_fit_time': array([5.45474523e-05, 2.60767614e-04, 2.55919884e-05, 8.02475996e-05,
        2.26618918e-05, 6.92919484e-06]),
 'mean_score_time': array([0.00075006, 0.00068831, 0.00015696, 0.00020059, 0.00017428,
        0.00015728]),
 'std_score_time': array([3.83495871e-05, 3.42673498e-04, 1.26643936e-05, 8.37826392e-05,
        1.02621590e-05, 6.68800828e-06]),
 'param_max_depth': masked_array(data=[1, 1, 2, 2, 3, 3],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_min_samples_split': masked_array(data=[2, 3, 2, 3, 2, 3],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 1, 'min_samples_split': 2},
  {'max_depth': 1, 'min_samples_split': 3},
  {'max_depth': 2, 'min_samples_split': 2},
  {'max_depth': 2, 'min_sa

In [59]:
print('GridSearchCV 최적 파라미터: ', grid_dtree.best_params_)
print(f'GridSearchCV 최고 정확도: {grid_dtree.best_score_}')

# refit=True이기 때문에 학습이 완료된 Estimator을 내포하고 있으므로 predict로 바로 예측 가능
pred = grid_dtree.predict(X_test)
print(f'테스트 데이터 세트 정확도 : {np.round(accuracy_score(y_test, pred), 4)}')

GridSearchCV 최적 파라미터:  {'max_depth': 3, 'min_samples_split': 2}
GridSearchCV 최고 정확도: 0.975
테스트 데이터 세트 정확도 : 0.9667


In [60]:
# refit으로 이미 학습된 estimator
estimator = grid_dtree.best_estimator_

pred = estimator.predict(X_test)
print(f'테스트 데이터 세트 정확도 : {np.round(accuracy_score(y_test, pred), 4)}')

테스트 데이터 세트 정확도 : 0.9667
